# Optimization results visualization

In [ ]:
# TODO :
# - Couleurs graphes
# - Date en axe x
# - Trouver la liste des chemins fichiers excels des résultats

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"

### Loading the results data

In [88]:
Tmax = 168 #optimization for 1 week (7*24=168 hours)
Tmaxmax = Tmax + 24


def load_assets():
    return pd.read_excel('./inputs_test.xlsx', sheet_name="ASSETS").set_index('Name')

def load_timeseries():
    return pd.read_excel('./inputs_test.xlsx', sheet_name="TIMESERIES")

def read_results():
    list_p_out = []
    list_p_in = []
    list_stock_e = []
    list_assets_state = []

    excel_files = ['./results_semaine_0.xlsx'] #, './results_semaine_1.xlsx']
    for week, excel_file in enumerate(excel_files) :
        for (sheet, list_df) in [("P_OUT", list_p_out), ("P_IN", list_p_in), ("STOCK_E", list_stock_e)]:
            df = pd.read_excel(excel_file, sheet_name=sheet)
            df["Time"] = df["Time"] + Tmax*week
            df = df.set_index("Time")
            list_df.append(df)
        list_assets_state.append(pd.read_excel(excel_file, sheet_name="ASSETS_STATE").set_index('asset_name'))
    
    # Fusionner les résultats des différentes semaines et enlever les valeurs des anneaux de garde
    df_p_out = pd.concat(list_p_out).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_p_in = pd.concat(list_p_in).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_stock_e = pd.concat(list_stock_e).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_assets_state = pd.concat(list_assets_state)

    return df_p_out, df_p_in, df_stock_e, df_assets_state

In [89]:
df_p_out, df_p_in, df_stock_e, df_assets_state = read_results()

In [91]:
df_assets = load_assets()
df_assets

,energy_in,energy_out,region,avail,pmax,pmin,dmin,on_init,h_on,h_off,efficiency,emax,e_init,price,family
Name,,,,,,,,,,,,,,,
Methaniseur,NaN,gas,s,1,10000,100,10,0,0,10,1.0,0,0,0,disp
CCG,gas,elec,n,1,10000,100,8,0,0,15,0.8,0,0,10,disp
STEP,elec,elec,n,1,10000,0,0,0,0,0,0.5,50000,200,0,stock


In [92]:
df_timeseries = load_timeseries()
df_timeseries.index = df_timeseries.index + 1
df_timeseries = df_timeseries.loc[:len(df_p_out)]

### Building graphs

In [122]:
for energy_vector, region in [('gas', 'n'), ('gas', 's'), ('h2', 'n'), ('h2', 's'), ('elec', None)]:
    fig = go.Figure()

    # Fatal load/production
    region_filter = {'n': 'NORTH', 's': 'SOUTH', None: ''}
    for fatal in [col for col in df_timeseries.columns if (((energy_vector).upper() in col) and (region_filter[region] in col))]:        
        if 'LOAD' in fatal :
            fig.add_trace(go.Bar(
                    x=df_timeseries.index,
                    y=-df_timeseries[fatal],
                    base=0,
                    name=f"{fatal}"
                ))
        else :
            fig.add_trace(go.Bar(
                    x=df_timeseries.index,
                    y=df_timeseries[fatal],
                    base=0,
                    name=f"P_out {fatal}"
                ))

    # Consuming assets
    for asset in df_assets[(df_assets['energy_in']==energy_vector) & (region is None or df_assets['region']==region)].index:
        fig.add_trace(go.Bar(
                x=df_p_in.index,
                y=-df_p_in[asset],
                base=0,
                name=f"P_in {asset}"
            ))

    # Producing assets
    for asset in df_assets[(df_assets['energy_out']==energy_vector) & (region is None or df_assets['region']==region)].index:
        fig.add_trace(go.Bar(
                x=df_p_out.index,
                y=df_p_out[asset],
                base=0,
                name=f"P_out {asset}"
            ))
        
    # Storage assets
    for storage in df_assets[(df_assets['energy_out']==energy_vector) & (df_assets['family']=='stock') & (region is None or df_assets['region']==region)].index:
        fig.add_trace(go.Scatter(
                x=df_p_out.index,
                y=100 * df_stock_e[storage]/df_assets['emax'][storage],
                name=f"SOC {storage}",
                mode='lines', yaxis="y2"
            ))
        
    # Imports/exports
    other_region = {'n':'s', 's':'n'}
    if region :
        fig.add_trace(go.Bar(
                x=df_p_in.index,
                y=df_p_in[f"{energy_vector.capitalize()}_interconnexion_{other_region[region].upper()}"],
                base=0,
                name=f"P_in {energy_vector.capitalize()}_interconnexion_{other_region[region].upper()}"
            ))
        fig.add_trace(go.Bar(
                x=df_p_out.index,
                y=-df_p_out[f"{energy_vector.capitalize()}_interconnexion_{other_region[region].upper()}"],
                base=0,
                name=f"P_out {energy_vector.capitalize()}_interconnexion_{other_region[region].upper()}"
            ))


    fig.update_layout(
        title=f"{energy_vector.capitalize()} management in {region_filter[region].capitalize()} Zootopia",
        xaxis_title="Time",
        yaxis=dict(
            title="Power [MW]",
            side="left",
            ticksuffix=" MW"
        ),
        yaxis2=dict(
            title="State Of Charge [%]",
            overlaying="y",
            side="right",
            showgrid=False,
            range=[0, 100],
            ticksuffix=" %"
        ),
        barmode='stack',
        template = 'plotly_dark',
        bargap = 0,
    )

    fig.show()